In [1]:
%matplotlib inline
import cProfile
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.stats import truncnorm
import timeit

In [2]:
image_size = 28 # width and length
no_of_different_labels = 10 #  i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size
data_path = "./data/"

## Load and visualize the data

In [3]:
test_data = pd.read_csv(data_path + "mnist_test.1k.csv", delimiter=",").values

fac = 0.99 / 255
test_imgs = np.asfarray(test_data[:, 1:], dtype=np.float32) * fac + 0.01
test_imgs = test_imgs.reshape(test_imgs.shape[0], 1, test_imgs.shape[1])

test_labels = np.asfarray(test_data[:, :1], dtype=np.float32)

lr = np.arange(no_of_different_labels)
# transform labels into one hot representation
test_labels_one_hot = (lr==test_labels).astype(np.float32)

# we don't want zeroes and ones in the labels neither:
test_labels_one_hot[test_labels_one_hot==0] = 0.001
test_labels_one_hot[test_labels_one_hot==1] = 0.999

In [5]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [6]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [7]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [8]:
class TanhLayer(ActivationLayer):
    # static
    e = 2.71828182845904523536028747135266249775724709369995
    
    #http://www.plunk.org/~hatch/rightway.php
    #https://math.stackexchange.com/questions/518758/alternative-form-for-sinhx-coshx
    @staticmethod
    def tanh(x):   
        e = TanhLayer.e
        return (1 - e ** (-2 * x)) / (1 + e ** (-2 * x)) 
        #return (1-np.exp(-2 * x))/(1+np.exp(-2 * x))

    @staticmethod
    def tanh_prime(x):
        return 1-TanhLayer.tanh(x)**2
    
    def __init__(self):
        super(TanhLayer,self).__init__(self.tanh, self.tanh_prime)
    

In [9]:
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [10]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
    
    def save(self, fname):
        import pickle
        with open(fname, "bw") as fh:
            pickle.dump(self, fh)

    @classmethod
    def load(cls, fname):
        import pickle
        with open(fname, "br") as fh:
            return pickle.load(fh)

In [11]:
net = Network.load('network.pkl')

In [12]:
for i in range(len(net.layers)):
    l = net.layers[i]
    is_fc = isinstance(l, FCLayer)    
    if is_fc:
        print ("Index: " + str(i))
        print ("Layer Name: " + str(l.__class__.__name__))
        print ("Layer is FC: " + str(is_fc))
        weights = l.weights
        print ("Weights: ", weights.shape)

    print ()

Index: 0
Layer Name: FCLayer
Layer is FC: True
Weights:  (784, 80)


Index: 2
Layer Name: FCLayer
Layer is FC: True
Weights:  (80, 40)


Index: 4
Layer Name: FCLayer
Layer is FC: True
Weights:  (40, 20)


Index: 6
Layer Name: FCLayer
Layer is FC: True
Weights:  (20, 10)



In [27]:
layer_id = 6
layer = net.layers[layer_id]
weights = layer.weights
bias = layer.bias
inputs = layer.input
outputs = layer.output
print ("Weights: ", weights.shape)
print ("Bias: ", bias.shape)
print ("Inputs: ", inputs.shape)
print ("Outputs: ", outputs.shape)
print ()

# flatten the bias, input and outputs matricies into arrays
bias = bias.flatten()
inputs = inputs.flatten()
outputs = outputs.flatten()

print ("Weights: ", weights.shape)
print ("Bias: ", bias.shape)
print ("Inputs: ", inputs.shape)
print ("Outputs: ", outputs.shape)

Weights:  (20, 10)
Bias:  (1, 10)
Inputs:  (1, 20)
Outputs:  (1, 10)

Weights:  (20, 10)
Bias:  (10,)
Inputs:  (20,)
Outputs:  (10,)


In [28]:
# how its done in dot.sv
def dot(weights,inputs):
    outs = np.zeros(weights.shape[1], dtype=np.float32)
    for i in range(weights.shape[0]): # input length
        for j in range(weights.shape[1]): # output length
            outs[j] = outs[j] + weights[i][j] * inputs[i]
    return outs

# my results
print (dot(weights,inputs) + bias)
# reference results
print (outputs)

[ 4.41362254e-03  2.98508890e-03 -2.72925311e-03  1.12005720e-03
  1.34415123e-04  5.53221731e-03  1.00080436e+00 -1.55906530e-03
 -1.35419525e-03 -1.24714277e-04]
[ 4.41370127e-03  2.98510492e-03 -2.72925683e-03  1.12005489e-03
  1.34415926e-04  5.53221199e-03  1.00080440e+00 -1.55908733e-03
 -1.35418249e-03 -1.24661481e-04]


In [43]:
# Now let's create some Verilog arrays
import numpy as np
import struct

def flt_to_hex(f):
    return hex(struct.unpack('<I', struct.pack('<f', f))[0])

file_name = "sim_last_layer.ipynb"

In [44]:
# Now let's the weight matrix to Verilog 
print ('// See %s for more details' % file_name)
print ('localparam ROWS = %d;' %weights.shape[0]) # Input Size
print ('localparam COLS = %d;' %weights.shape[1]) # Output Size
print ()
print ('//weights:\n//' + str(weights).replace('\n', '\n//'))

print ('localparam [31:0] weights [0:ROWS-1] [0:COLS-1] = \'{')
for i in range(weights.shape[0]):
    flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], weights[i]))
    print ('\t\'{' + ','.join(flts_hex)  + '}', end='')
    print (',' if i < weights.shape[0]-1 else '')
print ('};')

// See sim_last_layer.ipynb for more details
localparam ROWS = 20;
localparam COLS = 10;

//weights:
//[[ 2.58326735e-02  3.10679128e-02  7.49541483e-02  6.94312717e-02
//   2.79411147e-02 -3.88900762e-01  3.34132064e-02  2.34185058e-02
//   6.41438352e-02  3.90812610e-02]
// [ 4.74299141e-01 -3.07274602e-02 -7.55915272e-02 -6.56207528e-02
//  -2.70176780e-02 -1.19249905e-01 -3.36554767e-02 -2.31518535e-02
//  -6.25688793e-02 -3.85192568e-02]
// [ 1.10262082e-01 -2.19398860e-01  1.35315516e-01  3.27390767e-02
//  -1.00836485e-01  3.15356007e-02 -7.07275928e-02 -6.07912470e-02
//   1.09173516e-01 -9.76181188e-02]
// [-9.52519917e-02 -1.53209934e-02  2.77056965e-01  2.98086684e-01
//  -7.57019013e-02 -3.56717738e-01 -3.63614657e-01 -1.97010157e-01
//   7.03833589e-02 -6.57219942e-02]
// [-5.40141227e-03 -1.75347015e-03 -2.18014605e-02 -4.35117789e-02
//   1.26061503e-02 -6.80273724e-02 -2.23201639e-02  5.77344579e-02
//   7.78135215e-04  9.30981529e-02]
// [ 1.55772139e-01  3.44770828e-0

In [46]:
# bias vector
flts = bias
flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], flts))
flts_str = list(map('{:+2.9f}'.format, flts))
offset=6
print ('// See %s for more details' % file_name)
print ('parameter NUM_VALUES = %d;' % len(flts))
print ('parameter [31:0] VALUES [0:NUM_VALUES-1] = {')
for i in range(0, len(flts), offset):
    print ('//' + ', '.join(flts_str[i:i+offset]), end='\n')
    print ('  ' + ', '.join(flts_hex[i:i+offset] ), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 

// See sim_last_layer.ipynb for more details
parameter NUM_VALUES = 10,
parameter [31:0] VALUES [0:NUM_VALUES-1] = {
//+0.399316686, +0.458808510, +0.334676876, +0.464973744, +0.324586756, -0.288951742
  32'h3ecc733d, 32'h3eeae8f3, 32'h3eab5ac4, 32'h3eee110a, 32'h3ea6303c, 32'hbe93f17c,
//-0.214477124, +0.149038328, +0.169016160, +0.133995899
  32'hbe5b9fe4, 32'h3e189d81, 32'h3e2d1292, 32'h3e093639 
};


In [47]:
# input values 
flts = inputs
flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], flts))
flts_str = list(map('"{:2.4f}"'.format, flts))
offset=6
print ('static bit [31:0] fpHex [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_hex[i:i+offset] ), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 
print ('static string fpAscii [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_str[i:i+offset]), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};')
print ('static int MAX_SIZE = %d;' % len(flts)) 

static bit [31:0] fpHex [0:19] = {
 32'h3f7fcc9b, 32'hbf7fce23, 32'hbf7f2897, 32'hbf7fb914, 32'hbf7fcae6, 32'hbf7fcd23,
 32'hbf7ff7cb, 32'hbf7fffe4, 32'hbf7ffed8, 32'hbf7fd52c, 32'hbf7fffef, 32'hbf7fd8e1,
 32'h3f7ffb91, 32'hbf7ffd7b, 32'h3f7f86a0, 32'h3f7feeb1, 32'h3f7f27b7, 32'h3f7f52d6,
 32'hbf7fc58d, 32'hbf7e119e 
};
static string fpAscii [0:19] = {
 "0.9992", "-0.9992", "-0.9967", "-0.9989", "-0.9992", "-0.9992",
 "-0.9999", "-1.0000", "-1.0000", "-0.9993", "-1.0000", "-0.9994",
 "0.9999", "-1.0000", "0.9981", "0.9997", "0.9967", "0.9974",
 "-0.9991", "-0.9925" 
};
static int MAX_SIZE = 20;


In [48]:
# output buffer

LUT_LIM = 2 # seems to be flattened
LUT_SIZE = 64 # 64 yields 95.6% accuracy
LUT_STEP = ((LUT_LIM+LUT_LIM)/LUT_SIZE)
LUT_XS = ( np.arange(-LUT_LIM, LUT_LIM, step=LUT_STEP) + 
            np.arange(LUT_LIM, -LUT_LIM, step=-LUT_STEP)[::-1] ) /2
LUT_YS = np.array( list((map( lambda x: 
                            np.tanh(x), 
                            LUT_XS))), dtype=np.float32)
X_SHIFT = int(LUT_LIM * (2<<15))
X_MAX = int( LUT_XS[-1] * (2<<15)) + X_SHIFT
def vlog_tanh(x):
    x_int = int(x * (2<<15)) & (2**32-1)    
    x_scaled = (x_int + X_SHIFT) & (2**32-1)
    lut_idx = x_scaled >> 12    
    if (lut_idx & X_MAX): return LUT_YS[0]
    elif (lut_idx >= LUT_SIZE): return LUT_YS[-1]
    else:  return LUT_YS[lut_idx] 

dots = dot(weights,inputs)
b_dots = dots + bias 
outs = list(map(vlog_tanh, b_dots))

flts = outs
flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], flts))
flts_str = list(map('"{:2.4f}"'.format, flts))
offset=6
print ('static bit [31:0] fpHex [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_hex[i:i+offset] ), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 
print ('static string fpAscii [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_str[i:i+offset]), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 
print ('static int MAX_SIZE = %d;' % len(flts))

static bit [31:0] fpHex [0:9] = {
 32'h3cffeaad, 32'h3cffeaad, 32'hbcffeaad, 32'h3cffeaad, 32'h3cffeaad, 32'h3cffeaad,
 32'h3f463fae, 32'hbcffeaad, 32'hbcffeaad, 32'hbcffeaad 
};
static string fpAscii [0:9] = {
 "0.0312", "0.0312", "-0.0312", "0.0312", "0.0312", "0.0312",
 "0.7744", "-0.0312", "-0.0312", "-0.0312" 
};
static int MAX_SIZE = 10;
